Author: Nicolo' Sgobba

Task: Explore and cluster the neighborhoods in Toronto.
Wikipedia Page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [166]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

from bs4 import BeautifulSoup

print('All libraries imported successfully.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
All libraries imported successfully.


In [167]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')

My_table = soup.find('table',{'class':'wikitable sortable'})

In [169]:
lst_dict = []

for items in My_table.find_all('tr')[1::1]:
    data = items.find_all(['th','td'])

    try:
        Postcode = data[0].text
        Borough = data[1].text
        Neighbourhood = data[2].text
    except IndexError:pass
    
    lst_dict.append({'Postcode':Postcode, 'Borough':Borough, 'Neighbourhood': Neighbourhood})

table = pd.DataFrame.from_dict(lst_dict)

In [173]:
TempTable = table[table.Borough != 'Not assigned']

In [171]:
TempTable.Neighbourhood[8] = "Queen's Park"
TempTable = TempTable.replace('\n', '', regex=True)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2862: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [174]:
temp = TempTable.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [175]:
pd.set_option('expand_frame_repr', False)

print(temp)

    Postcode           Borough                                      Neighbourhood
0        M1B       Scarborough                                 Rouge\n, Malvern\n
1        M1C       Scarborough       Highland Creek\n, Rouge Hill\n, Port Union\n
2        M1E       Scarborough            Guildwood\n, Morningside\n, West Hill\n
3        M1G       Scarborough                                           Woburn\n
4        M1H       Scarborough                                        Cedarbrae\n
5        M1J       Scarborough                              Scarborough Village\n
6        M1K       Scarborough  East Birchmount Park\n, Ionview\n, Kennedy Park\n
7        M1L       Scarborough              Clairlea\n, Golden Mile\n, Oakridge\n
8        M1M       Scarborough  Cliffcrest\n, Cliffside\n, Scarborough Village...
9        M1N       Scarborough                    Birch Cliff\n, Cliffside West\n
10       M1P       Scarborough  Dorset Park\n, Scarborough Town Centre\n, Wexf...
11       M1R    

In [176]:
temp.shape

(103, 3)

In [177]:
url="https://cocl.us/Geospatial_data"
code=pd.read_csv(url)
code=code.rename(index=str, columns={'Postal Code': 'Postcode'})
code.head

<bound method NDFrame.head of     Postcode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476
5        M1J  43.744734 -79.239476
6        M1K  43.727929 -79.262029
7        M1L  43.711112 -79.284577
8        M1M  43.716316 -79.239476
9        M1N  43.692657 -79.264848
10       M1P  43.757410 -79.273304
11       M1R  43.750072 -79.295849
12       M1S  43.794200 -79.262029
13       M1T  43.781638 -79.304302
14       M1V  43.815252 -79.284577
15       M1W  43.799525 -79.318389
16       M1X  43.836125 -79.205636
17       M2H  43.803762 -79.363452
18       M2J  43.778517 -79.346556
19       M2K  43.786947 -79.385975
20       M2L  43.757490 -79.374714
21       M2M  43.789053 -79.408493
22       M2N  43.770120 -79.408493
23       M2P  43.752758 -79.400049
24       M2R  43.782736 -79.442259
25       M3A  43.753259 -79.329656
26       M3B  43.745906 -

In [178]:
neighborhoods = temp.merge(code, on='Postcode', how='left')
neighborhoods.shape

(103, 5)

In [179]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [180]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [185]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)

In [186]:
map_Toronto

In [188]:
#CLIENT_ID = 'your-client-ID' # your Foursquare ID
CLIENT_ID = 'OHOJGBQX3OYUYLZLB53Z2DIRCUPNMYDD4CXKUJNWWMP12H15' # your Foursquare ID

#CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
CLIENT_SECRET = 'TPO3OZUZX2ABG5XUJ4P5LCTCYWKWZGEVVNS1O1PGLVE305OM' # your Foursquare Secret

VERSION = '20180604'
LIMIT = 30

print('Credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentials:
CLIENT_ID: OHOJGBQX3OYUYLZLB53Z2DIRCUPNMYDD4CXKUJNWWMP12H15
CLIENT_SECRET:TPO3OZUZX2ABG5XUJ4P5LCTCYWKWZGEVVNS1O1PGLVE305OM


In [190]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
        
        for i in range(1,50):
            current_radius = radius + 50 * (i - 1)
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                current_radius, 
                LIMIT)

            # make the GET request
            groups = requests.get(url).json()["response"]['groups']

            selected_group = []
            for group in groups:
                if len(group) > len(selected_group):
                    selected_group = group
                  

            venue_list = []
            if len(selected_group) > 0:
                results = selected_group['items']

                # return only relevant information for each nearby venue
                venue_list = [(
                    name, 
                    lat, 
                    lng, 
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name'],
                    current_radius
                ) for v in results if v['venue']['categories']]
            
            if len(venue_list) > 10:
                venues_list.append(venue_list)
                break
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Explore Radius']
    
    return(nearby_venues)

In [197]:
Toronto_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

KeyError: 'groups'

In [193]:
print(Toronto_venues.shape)

(1804, 8)


In [194]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Explore Radius
Neighborhood,,,,,,,
"Adelaide\n, King\n, Richmond\n",30,30,30,30,30,30,30
Agincourt\n,15,15,15,15,15,15,15
"Agincourt North\n, L'Amoreaux East\n, Milliken\n, Steeles East\n",15,15,15,15,15,15,15
"Albion Gardens\n, Beaumond Heights\n, Humbergate\n, Jamestown\n, Mount Olive\n, Silverstone\n, South Steeles\n, Thistletown\n",11,11,11,11,11,11,11
"Alderwood\n, Long Branch\n",11,11,11,11,11,11,11
"Bathurst Manor\n, Downsview North\n, Wilson Heights\n",19,19,19,19,19,19,19
Bayview Village\n,12,12,12,12,12,12,12
"Bedford Park\n, Lawrence Manor East\n",25,25,25,25,25,25,25
Berczy Park\n,30,30,30,30,30,30,30


In [196]:
Toronto_venues.groupby('Neighborhood').count().shape

NameError: name 'Toronto_venues' is not defined

In [145]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 260 uniques categories.


In [146]:
Toronto_onehot = pd.get_dummies(Tornto_venues[['Venue Category']], prefix="", prefix_sep="")
Toronto_onehot['Neighborhood'] = Tornto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

In [147]:
#Toronto_onehot.shape

(1804, 260)

In [148]:
toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [149]:
#toronto_grouped.shape

(103, 260)

In [150]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print(hood)
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0           Steakhouse  0.10
1  American Restaurant  0.10
2     Asian Restaurant  0.07
3                Hotel  0.07
4                 Café  0.07


----Agincourt----
                      venue  freq
0            Breakfast Spot  0.07
1          Sushi Restaurant  0.07
2              Skating Rink  0.07
3                 Pool Hall  0.07
4  Mediterranean Restaurant  0.07


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                  venue  freq
0             BBQ Joint  0.13
1           Pizza Place  0.13
2          Noodle House  0.07
3  Fast Food Restaurant  0.07
4  Caribbean Restaurant  0.07


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.18
1       Sandwich Place  0.09
2  Fried Chicken Joint  0.09
3          Pizza Place  0.09
4       Discount Store  0.09


----Alderwood, Long Branch

                venue  freq
0      Discount Store  0.18
1  Light Rail Station  0.09
2       Train Station  0.09
3          Hobby Shop  0.09
4        Hockey Arena  0.09


----East Toronto----
               venue  freq
0               Park  0.18
1  Convenience Store  0.09
2               Café  0.09
3             Hostel  0.09
4        Pizza Place  0.09


----Emery, Humberlea----
              venue  freq
0       Coffee Shop  0.18
1       Golf Course  0.18
2    Discount Store  0.09
3  Storage Facility  0.09
4            Bakery  0.09


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.13
1           Coffee Shop  0.07
2  Fast Food Restaurant  0.03
3              Tea Room  0.03
4            Food Court  0.03


----First Canadian Place, Underground city----
           venue  freq
0  Deli / Bodega  0.10
1           Café  0.10
2            Bar  0.07
3     Restaurant  0.07
4     Steakhouse  0.07


----Flemingdon Park, Don Mills South----
               

            venue  freq
0            Café  0.13
1     Coffee Shop  0.07
2  Clothing Store  0.07
3           Plaza  0.03
4   Shopping Mall  0.03


----Scarborough Village----
               venue  freq
0     Ice Cream Shop  0.18
1     Sandwich Place  0.09
2      Bowling Alley  0.09
3  Convenience Store  0.09
4        Pizza Place  0.09


----Silver Hills, York Mills----
                    venue  freq
0                    Park  0.23
1  Furniture / Home Store  0.08
2          Discount Store  0.08
3       Convenience Store  0.08
4             Coffee Shop  0.08


----St. James Town----
                 venue  freq
0          Coffee Shop  0.13
1   Italian Restaurant  0.10
2            Gastropub  0.10
3  Japanese Restaurant  0.07
4                Hotel  0.07


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0        Cocktail Bar  0.10
1                Café  0.10
2  Seafood Restaurant  0.07
3      Farmers Market  0.07
4          Steakhouse  0.03


----Studio District----
  

In [151]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [152]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",American Restaurant,Steakhouse,Asian Restaurant,Hotel,Café,Pizza Place,Speakeasy,Smoke Shop,Seafood Restaurant,Plaza
1,Agincourt,Lounge,Breakfast Spot,Discount Store,Skating Rink,Sushi Restaurant,Motorcycle Shop,Supermarket,Mediterranean Restaurant,Pool Hall,Seafood Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",BBQ Joint,Pizza Place,Fast Food Restaurant,Gym,Park,Noodle House,Chinese Restaurant,Caribbean Restaurant,Bubble Tea Shop,Shop & Service
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Beer Store,Discount Store,Sandwich Place,Japanese Restaurant,Fast Food Restaurant,Coffee Shop,Pharmacy
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Pool,Sandwich Place,Bank,Gas Station,Pub,Skating Rink,Gym,Coffee Shop
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pharmacy,Frozen Yogurt Shop,Bridal Shop,Sandwich Place,Diner,Middle Eastern Restaurant,Bank,Supermarket,Restaurant
6,Bayview Village,Japanese Restaurant,Bank,Grocery Store,Chinese Restaurant,Café,Restaurant,Shopping Mall,Skate Park,Skating Rink,Pharmacy
7,"Bedford Park, Lawrence Manor East",Italian Restaurant,Fast Food Restaurant,Coffee Shop,Thai Restaurant,Pet Store,Pharmacy,Comfort Food Restaurant,Pub,Café,Butcher
8,Berczy Park,Cocktail Bar,Bakery,Café,Seafood Restaurant,Farmers Market,Park,Steakhouse,Jazz Club,Basketball Stadium,Beer Bar
9,"Birch Cliff, Cliffside West",College Stadium,Park,Skating Rink,Bank,Discount Store,Diner,Café,General Entertainment,Thai Restaurant,Convenience Store


In [153]:
# number of clusters
kclusters = 10
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_.shape

(103,)

In [154]:
toronto_merged = neighborhoods

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each Neighbourhood neighborhood neighborhoods
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,3,Fast Food Restaurant,Coffee Shop,Filipino Restaurant,Hobby Shop,Bus Station,Martial Arts Dojo,Paper / Office Supplies Store,Spa,African Restaurant,Construction & Landscaping
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3,Grocery Store,Hotel,Pharmacy,Park,Burger Joint,Breakfast Spot,Pizza Place,Italian Restaurant,Gym,Hardware Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Pizza Place,Medical Center,Fried Chicken Joint,Rental Car Location,Electronics Store,Mexican Restaurant,Breakfast Spot,Sports Bar,Greek Restaurant,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Indian Restaurant,Fast Food Restaurant,Chinese Restaurant,Department Store,Bakery,Supplement Shop,Juice Bar,Electronics Store,Park
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Indian Restaurant,Hakka Restaurant,Chinese Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Flower Shop


In [155]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [ ]:
map_clusters

In [156]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,0,Coffee Shop,Indian Restaurant,Fast Food Restaurant,Chinese Restaurant,Department Store,Bakery,Supplement Shop,Juice Bar,Electronics Store,Park
4,Scarborough,0,Indian Restaurant,Hakka Restaurant,Chinese Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Flower Shop
24,North York,0,Coffee Shop,Pharmacy,Pizza Place,Eastern European Restaurant,Bus Line,Asian Restaurant,Bakery,Bank,Sandwich Place,Discount Store
53,Downtown Toronto,0,Coffee Shop,Bakery,Park,Breakfast Spot,Mexican Restaurant,Pub,Spa,Café,Restaurant,Italian Restaurant
59,Downtown Toronto,0,Hotel,Park,Café,Sports Bar,Brewery,Ice Cream Shop,Bakery,Supermarket,Italian Restaurant,Skating Rink
61,Downtown Toronto,0,Café,Restaurant,Deli / Bodega,Hotel,Gym,Gastropub,Steakhouse,Bakery,Japanese Restaurant,Ice Cream Shop
69,Downtown Toronto,0,Cocktail Bar,Café,Seafood Restaurant,Farmers Market,Belgian Restaurant,Fountain,Food Truck,Steakhouse,Jazz Club,Beer Bar
92,Etobicoke,0,Gym,Convenience Store,Burger Joint,Fast Food Restaurant,Sandwich Place,Supplement Shop,Thrift / Vintage Store,Bakery,Discount Store,Flower Shop
100,Etobicoke,0,Pharmacy,American Restaurant,Mobile Phone Shop,Bank,Supermarket,Sandwich Place,Chinese Restaurant,Intersection,Beer Store,Coffee Shop


In [157]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1,Pizza Place,Medical Center,Fried Chicken Joint,Rental Car Location,Electronics Store,Mexican Restaurant,Breakfast Spot,Sports Bar,Greek Restaurant,Fast Food Restaurant
25,North York,1,Park,Bus Stop,Furniture / Home Store,Food & Drink Shop,Pizza Place,Road,Convenience Store,Caribbean Restaurant,Train Station,Dessert Shop
46,Central Toronto,1,Sporting Goods Shop,Coffee Shop,Clothing Store,Fast Food Restaurant,Diner,Salon / Barbershop,Rental Car Location,Sandwich Place,Mexican Restaurant,Chinese Restaurant
47,Central Toronto,1,Dessert Shop,Sandwich Place,Pizza Place,Café,Coffee Shop,Sushi Restaurant,Seafood Restaurant,Italian Restaurant,Diner,Farmers Market
72,North York,1,Grocery Store,Pizza Place,Bus Line,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Flower Shop,Fish Market,Latin American Restaurant,Metro Station
74,York,1,Park,Market,Mexican Restaurant,Women's Store,Gym,Pharmacy,Fast Food Restaurant,Japanese Restaurant,Sporting Goods Shop,Bakery
81,York,1,Pizza Place,Grocery Store,Video Game Store,Sandwich Place,Bus Line,Thai Restaurant,Furniture / Home Store,Convenience Store,Indian Restaurant,Park
90,Etobicoke,1,Sushi Restaurant,Park,Breakfast Spot,River,Burger Joint,Restaurant,Bakery,Dessert Shop,Bank,Bar
95,Etobicoke,1,Pizza Place,Park,Beer Store,Shopping Plaza,Liquor Store,Breakfast Spot,Electronics Store,Café,Grocery Store,Pharmacy


In [158]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Scarborough,2,Zoo Exhibit,Park,Restaurant,Dessert Shop,Golf Course,Grocery Store,Zoo,Vietnamese Restaurant,Deli / Bodega,Falafel Restaurant
36,East York,2,Spa,Cosmetics Shop,Skating Rink,Beer Store,Bus Line,Restaurant,Dance Studio,Curling Ice,Asian Restaurant,Video Store
38,East York,2,Sporting Goods Shop,Burger Joint,Coffee Shop,Bike Shop,Record Shop,Sushi Restaurant,Supermarket,Bank,Restaurant,Fish & Chips Shop
56,Downtown Toronto,2,Cocktail Bar,Bakery,Café,Seafood Restaurant,Farmers Market,Park,Steakhouse,Jazz Club,Basketball Stadium,Beer Bar
68,Downtown Toronto,2,Airport Terminal,Airport Service,Airport Lounge,Harbor / Marina,Airport Gate,Boat or Ferry,Sculpture Garden,Boutique,Airport Food Court,Plane
76,West Toronto,2,Discount Store,Supermarket,Bakery,Gym / Fitness Center,Furniture / Home Store,Park,Music Venue,Café,Middle Eastern Restaurant,Brewery
82,West Toronto,2,Café,Mexican Restaurant,Bar,Grocery Store,Antique Shop,Bakery,Italian Restaurant,Cajun / Creole Restaurant,Fried Chicken Joint,Furniture / Home Store
93,Etobicoke,2,Pharmacy,Playground,Park,Bakery,Café,Bank,Shopping Mall,Convenience Store,Golf Course,Skating Rink
102,Etobicoke,2,Coffee Shop,Casino,Hotel,Swiss Restaurant,Mediterranean Restaurant,Rental Car Location,Paper / Office Supplies Store,Sandwich Place,Dog Run,Racecourse


In [159]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3,Fast Food Restaurant,Coffee Shop,Filipino Restaurant,Hobby Shop,Bus Station,Martial Arts Dojo,Paper / Office Supplies Store,Spa,African Restaurant,Construction & Landscaping
1,Scarborough,3,Grocery Store,Hotel,Pharmacy,Park,Burger Joint,Breakfast Spot,Pizza Place,Italian Restaurant,Gym,Hardware Store
5,Scarborough,3,Ice Cream Shop,Coffee Shop,Train Station,Pizza Place,Bowling Alley,Sandwich Place,Japanese Restaurant,Restaurant,Fast Food Restaurant,Convenience Store
6,Scarborough,3,Discount Store,Hobby Shop,Bus Line,Department Store,Train Station,Chinese Restaurant,Hockey Arena,Light Rail Station,Convenience Store,Coffee Shop
7,Scarborough,3,Diner,Bakery,Bus Line,Ice Cream Shop,Metro Station,Bus Station,Intersection,Soccer Field,Park,Coworking Space
8,Scarborough,3,Pizza Place,Beach,Ice Cream Shop,Furniture / Home Store,Sports Bar,Park,Cajun / Creole Restaurant,Burger Joint,Zoo,Dumpling Restaurant
9,Scarborough,3,College Stadium,Park,Skating Rink,Bank,Discount Store,Diner,Café,General Entertainment,Thai Restaurant,Convenience Store
12,Scarborough,3,Lounge,Breakfast Spot,Discount Store,Skating Rink,Sushi Restaurant,Motorcycle Shop,Supermarket,Mediterranean Restaurant,Pool Hall,Seafood Restaurant
15,Scarborough,3,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Sandwich Place,Japanese Restaurant,Noodle House,Pizza Place,Pharmacy,Grocery Store,Coffee Shop
20,North York,3,Park,Gym,Pub,Intersection,Discount Store,Furniture / Home Store,Steakhouse,Convenience Store,Japanese Restaurant,Coffee Shop


In [160]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,North York,4,Clothing Store,Coffee Shop,Liquor Store,Theater,Bakery,Sporting Goods Shop,Burrito Place,Food Court,Japanese Restaurant,Department Store
34,North York,4,Coffee Shop,Park,Pizza Place,Lounge,Sporting Goods Shop,Portuguese Restaurant,Men's Store,Intersection,Hockey Arena,Zoo
99,Etobicoke,4,Pizza Place,Chinese Restaurant,Middle Eastern Restaurant,Sandwich Place,Ice Cream Shop,Discount Store,Supermarket,Intersection,Golf Driving Range,Coffee Shop


In [161]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,East Toronto,6,Sandwich Place,Pizza Place,Liquor Store,Sushi Restaurant,Italian Restaurant,Food & Drink Shop,Steakhouse,Fish & Chips Shop,Pet Store,Brewery
44,Central Toronto,6,Coffee Shop,Trail,Pharmacy,Gym / Fitness Center,Pool,Park,Restaurant,Café,Bus Line,Bookstore
62,North York,6,Italian Restaurant,Fast Food Restaurant,Coffee Shop,Thai Restaurant,Pet Store,Pharmacy,Comfort Food Restaurant,Pub,Café,Butcher
64,Central Toronto,6,Coffee Shop,Italian Restaurant,Park,Pharmacy,Gastropub,Café,Bakery,Bank,Asian Restaurant,Chinese Restaurant
70,Downtown Toronto,6,Deli / Bodega,Café,Restaurant,Coffee Shop,Steakhouse,Bar,American Restaurant,Art Gallery,Speakeasy,Food Court
78,West Toronto,6,Coffee Shop,Café,Breakfast Spot,Gym,Furniture / Home Store,Performing Arts Venue,Convenience Store,Climbing Gym,Nightclub,Caribbean Restaurant


In [162]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,North York,7,Park,Grocery Store,Vietnamese Restaurant,Shopping Mall,Bank,Tea Room,Moving Target,Plaza,Pizza Place,Coffee Shop
37,East Toronto,7,Bakery,Vegetarian / Vegan Restaurant,Pub,Trail,Electronics Store,Breakfast Spot,French Restaurant,Pet Store,Coffee Shop,Empanada Restaurant
60,Downtown Toronto,7,Coffee Shop,Café,Deli / Bodega,Restaurant,Bar,Japanese Restaurant,Bakery,Pub,Hotel Bar,Hotel


In [163]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,8,Indian Restaurant,Electronics Store,Vietnamese Restaurant,Gaming Cafe,Bakery,Chinese Restaurant,Brewery,Latin American Restaurant,Coffee Shop,Pet Store
11,Scarborough,8,Pizza Place,Vietnamese Restaurant,Bakery,Middle Eastern Restaurant,Korean Restaurant,Seafood Restaurant,Fish Market,Convenience Store,Breakfast Spot,Coffee Shop
13,Scarborough,8,Pizza Place,Pharmacy,Thai Restaurant,Chinese Restaurant,Noodle House,Bus Stop,Sandwich Place,Shopping Mall,Italian Restaurant,Fried Chicken Joint
17,North York,8,Pharmacy,Housing Development,Pizza Place,Chinese Restaurant,Recreation Center,Diner,Sandwich Place,Shopping Mall,Bank,Bakery
19,North York,8,Japanese Restaurant,Bank,Grocery Store,Chinese Restaurant,Café,Restaurant,Shopping Mall,Skate Park,Skating Rink,Pharmacy
30,North York,8,Coffee Shop,Turkish Restaurant,Park,Sandwich Place,Middle Eastern Restaurant,Latin American Restaurant,Italian Restaurant,Pizza Place,Vietnamese Restaurant,Airport
32,North York,8,Vietnamese Restaurant,Coffee Shop,Fast Food Restaurant,Supermarket,Baseball Field,Park,Falafel Restaurant,Grocery Store,Moving Target,Empanada Restaurant
40,East York,8,Park,Pizza Place,Convenience Store,Café,Restaurant,Caribbean Restaurant,Hostel,Beer Bar,Coffee Shop,Dim Sum Restaurant
41,East Toronto,8,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Grocery Store,Pub,Bookstore,Brewery,Bubble Tea Shop,Spa,Juice Bar
45,Central Toronto,8,Gym Pool,Sushi Restaurant,Convenience Store,Gourmet Shop,Park,Clothing Store,Breakfast Spot,Brewery,Asian Restaurant,Sandwich Place


In [165]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,9,BBQ Joint,Pizza Place,Fast Food Restaurant,Gym,Park,Noodle House,Chinese Restaurant,Caribbean Restaurant,Bubble Tea Shop,Shop & Service
